<a href="https://colab.research.google.com/github/sagar4real/CMP7005-PRAC1/blob/main/01_data_handling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Task 1 - DATA HANDLING**





### **1.1 Import Libraries**

In [ ]:
import pandas as pd
import numpy as np
import glob
import os


print("Libraries imported successfully")

Libraries imported successfully


### **1.2 Load and Inspect Individual CSV Files**

In [ ]:
# Folder where your CSVs are stored
DATA_FOLDER = "/content/CMP7005-PRAC1/data"

# Pattern to match your files
pattern = os.path.join(DATA_FOLDER, "*_data.csv")

# List all matching files
csv_files = sorted(glob.glob(pattern))

for(file) in csv_files:
    temp = pd.read_csv(file)
    print(file.split('/')[4] , " = " , temp.shape)

Ahmedabad_data.csv  =  (2009, 16)
Aizawl_data.csv  =  (113, 16)
Amaravati_data.csv  =  (951, 16)
Amritsar_data.csv  =  (1221, 16)
Bengaluru_data.csv  =  (2009, 16)
Bhopal_data.csv  =  (289, 16)
Brajrajnagar_data.csv  =  (938, 16)
Chandigarh_data.csv  =  (304, 16)
Chennai_data.csv  =  (2009, 16)
Coimbatore_data.csv  =  (386, 16)
Delhi_data.csv  =  (2009, 16)
Ernakulam_data.csv  =  (162, 16)
Gurugram_data.csv  =  (1679, 16)
Guwahati_data.csv  =  (502, 16)
Hyderabad_data.csv  =  (2006, 16)
Jaipur_data.csv  =  (1114, 16)
Jorapokhar_data.csv  =  (1169, 16)
Kochi_data.csv  =  (162, 16)
Kolkata_data.csv  =  (814, 16)
Lucknow_data.csv  =  (2009, 16)
Mumbai_data.csv  =  (2009, 16)
Patna_data.csv  =  (1858, 16)
Shillong_data.csv  =  (310, 16)
Talcher_data.csv  =  (925, 16)
Thiruvananthapuram_data.csv  =  (1112, 16)
Visakhapatnam_data.csv  =  (1462, 16)
merged_air_quality_data.csv  =  (118124, 16)


### **1.3 Merge All _data.csv Files**

In [ ]:
if len(csv_files) == 0:
    raise FileNotFoundError("No files found !")

# Load and append
df_list = []
for file in csv_files:
    temp = pd.read_csv(file)
    df_list.append(temp)

# Merge all into one DataFrame
merged_df = pd.concat(df_list, ignore_index=True)

print("Merged dataset in merged_df")


Merged dataset in merged_df


### **1.5 Basic Inspection of the Merged Dataset**

In [ ]:
# preview top rows
merged_df.head()

,City,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Ahmedabad,01/01/2015,NaN,NaN,0.92,18.22,17.15,NaN,0.92,27.64,133.36,0.00,0.02,0.00,NaN,NaN
1,Ahmedabad,02/01/2015,NaN,NaN,0.97,15.69,16.46,NaN,0.97,24.55,34.06,3.68,5.50,3.77,NaN,NaN
2,Ahmedabad,03/01/2015,NaN,NaN,17.40,19.30,29.70,NaN,17.40,29.07,30.70,6.80,16.40,2.25,NaN,NaN
3,Ahmedabad,04/01/2015,NaN,NaN,1.70,18.48,17.97,NaN,1.70,18.59,36.08,4.43,10.14,1.00,NaN,NaN
4,Ahmedabad,05/01/2015,NaN,NaN,22.10,21.42,37.76,NaN,22.10,39.33,39.31,7.01,18.89,2.78,NaN,NaN


In [ ]:
# check dataset structure
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147655 entries, 0 to 147654
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   City        147655 non-null  object 
 1   Date        147655 non-null  object 
 2   PM2.5       124665 non-null  float64
 3   PM10        91955 non-null   float64
 4   NO          129745 non-null  float64
 5   NO2         129730 non-null  float64
 6   NOx         126730 non-null  float64
 7   NH3         96015 non-null   float64
 8   CO          137360 non-null  float64
 9   SO2         128385 non-null  float64
 10  O3          127545 non-null  float64
 11  Benzene     119540 non-null  float64
 12  Toluene     107450 non-null  float64
 13  Xylene      57110 non-null   float64
 14  AQI         124250 non-null  float64
 15  AQI_Bucket  124250 non-null  object 
dtypes: float64(13), object(3)
memory usage: 18.0+ MB


In [ ]:
# check missing values
merged_df.isnull().sum().sort_values(ascending=False).head(20)

,0
Xylene,90545
PM10,55700
NH3,51640
Toluene,40205
Benzene,28115
AQI,23405
AQI_Bucket,23405
PM2.5,22990
NOx,20925
O3,20110


In [ ]:
# Summary statistics
merged_df.describe(include='all')


,City,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
count,147655,147655,124665.000000,91955.000000,129745.000000,129730.000000,126730.000000,96015.000000,137360.000000,128385.000000,127545.000000,119540.000000,107450.000000,57110.000000,124250.000000,124250
unique,26,2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
top,Ahmedabad,26/06/2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Moderate
freq,10045,130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44145
mean,NaN,NaN,67.450578,118.127103,17.574730,28.560659,32.309123,23.483476,2.248598,14.531977,34.491430,3.280840,8.700972,3.070128,166.463581,NaN
std,NaN,NaN,64.660412,90.603139,22.785495,24.474368,31.645512,25.683740,6.962783,18.133492,21.694588,15.810872,19.968792,6.323026,140.694320,NaN
min,NaN,NaN,0.040000,0.010000,0.020000,0.010000,0.000000,0.010000,0.000000,0.010000,0.010000,0.000000,0.000000,0.000000,13.000000,NaN
25%,NaN,NaN,28.820000,56.250000,5.630000,11.750000,12.820000,8.580000,0.510000,5.670000,18.860000,0.120000,0.600000,0.140000,81.000000,NaN
50%,NaN,NaN,48.570000,95.680000,9.890000,21.690000,23.520000,15.850000,0.890000,9.160000,30.840000,1.070000,2.970000,0.980000,118.000000,NaN
75%,NaN,NaN,80.590000,149.750000,19.950000,37.620000,40.130000,30.020000,1.450000,15.220000,45.570000,3.080000,9.150000,3.350000,208.000000,NaN


**Task 1 Report**

In this exercise I have downloaded the India Air Quality data set from Moodle and there were several csv files named ending in _data.csv. I wanted to use the same data for my analysis therefore I loaded each file into Jupyter Notebook and did an initial evaluation of the data.

First I made a list of all csv files (with the names of the csv files) available in the dataset folder. Then I loaded each one of these files separately. I then concatenated the individual files together into a single data set using **pandas.concat()** with ignore_index=True to prevent duplicate indexes.

Once the data had been merged, I conducted a preliminary assessment of the data using **.head()**, **.info()**, **.isnull().sum()** and **.describe()** to assess the structure of the data, the type of data in each column and how many missing values are present. The merged data set is now stored as **merged_df.csv** and will be the source data for the EDA in Task 2.